In [14]:
import numpy as np
import pandas as pd
import random
from scipy.spatial import distance

In [15]:
data = []
for userId in range(1,12):
    sample = {}
    sample["userId"] = userId
    for i in arr:
        sample[i] = random.choice([5,4,3,2,1,-1])
    data.append(sample)

In [16]:
df = pd.DataFrame(data)

In [17]:
df

,userId,Toy Story (1995),Grumpier Old Men (1995),Heat (1995),Seven (a.k.a. Se7en) (1995),"Usual Suspects, The (1995)",Forrest Gump (1994),Star Wars (1977)
0,1,5,2,3,5,1,4,5
1,2,4,2,2,-1,4,-1,-1
2,3,5,1,5,3,5,5,1
3,4,5,5,2,2,5,1,2
4,5,2,-1,-1,-1,2,1,5
5,6,-1,5,5,3,4,3,2
6,7,-1,2,-1,1,1,1,2
7,8,1,2,4,3,-1,5,2
8,9,-1,-1,1,-1,1,2,2
9,10,4,2,-1,2,3,4,-1


In [18]:
arr = ["Toy Story (1995)","Grumpier Old Men (1995)","Heat (1995)","Seven (a.k.a. Se7en) (1995)","Usual Suspects, The (1995)","Forrest Gump (1994)","Star Wars (1977)"]

In [60]:
import itertools

class RecSys:
    def __init__(self, k, data, userId, type):
        self.k = k
        self.userId = userId
        self.data = data
        self.results = {}
        self.top_k_results = {}
        self.type = type

    def preprocess(self):
        modified_data = {}
        if self.type == "User":
            for index, user in enumerate(self.data):
                values = list(user.values())
                modified_data[index + 1] = values[1:]
        elif self.type == "Item":
            modified_data = {movie: [user[movie] for user in self.data] for movie in self.data[0] if movie != 'userId'}
        self.modified_data = modified_data

    def cosine_similarity(self, a, b):
        return 1 - distance.cosine(a, b)

    def calculate_distance(self):
        self.preprocess()
        results = {}
        active_data = self.modified_data.pop(self.userId)
        for otherId, otherData in self.modified_data.items():
            indices_active_data = [index for index, value in enumerate(active_data) if value == -1]
            indices_other_data = [index for index, value in enumerate(otherData) if value == -1]
            union_indices = set(indices_active_data).union(indices_other_data)
            active_data_new = [value for index, value in enumerate(active_data) if index not in union_indices]
            other_data_new = [value for index, value in enumerate(otherData) if index not in union_indices]
            results[otherId] = self.cosine_similarity(active_data_new, other_data_new)
        self.results = results

    def top_k(self):
        self.calculate_distance()
        sorted_results = {k: v for k, v in sorted(self.results.items(), key=lambda item: item[1], reverse=True)}
        self.top_k_results = dict(itertools.islice(sorted_results.items(), self.k))
        return self.top_k_results

    def predict_rating(self,ItemName):
        avg = {}
        if self.type == "User":
            for i,j in self.top_k_results.items():
                if self.data[i-1][ItemName] == -1:
                    pass
                else:
                    print(self.data[i-1])
                    avg[j]= self.data[i-1][ItemName]
        if self.type == "Item":
            for i,j in self.top_k_results.items():
                    print(i,j)
                    if self.data[ItemName - 1][i] == -1:
                        pass
                    else:
                        avg[j]= self.data[ItemName - 1][i]
        print(avg)
        weights = list(avg.keys())
        values = list(avg.values())
        weighted_avg = sum(w*v for w, v in zip(weights, values)) / sum(weights)
        return weighted_avg


In [53]:
rs = RecSys(3,data,2,'User')

In [54]:
rs.top_k()

{5: 1, 11: 0.9908673886137246, 10: 0.9903751369442766}

In [55]:
rs.predict_rating("Seven (a.k.a. Se7en) (1995)")

{'userId': 11, 'Toy Story (1995)': 3, 'Grumpier Old Men (1995)': 2, 'Heat (1995)': 2, 'Seven (a.k.a. Se7en) (1995)': 4, 'Usual Suspects, The (1995)': 4, 'Forrest Gump (1994)': 3, 'Star Wars (1977)': 1}
{'userId': 10, 'Toy Story (1995)': 4, 'Grumpier Old Men (1995)': 2, 'Heat (1995)': -1, 'Seven (a.k.a. Se7en) (1995)': 2, 'Usual Suspects, The (1995)': 3, 'Forrest Gump (1994)': 4, 'Star Wars (1977)': -1}


3.000248456038621

In [61]:
res = RecSys(4,data,"Usual Suspects, The (1995)","Item")

In [62]:
res.top_k()

{'Toy Story (1995)': 0.9223780407186633,
 'Heat (1995)': 0.8838834764831843,
 'Grumpier Old Men (1995)': 0.8639153069476767,
 'Forrest Gump (1994)': 0.8366451529746755}

In [63]:
res.predict_rating(2)

Toy Story (1995) 0.9223780407186633
Heat (1995) 0.8838834764831843
Grumpier Old Men (1995) 0.8639153069476767
Forrest Gump (1994) 0.8366451529746755
{0.9223780407186633: 4, 0.8838834764831843: 2, 0.8639153069476767: 2}


2.6908741266694567